# Natural Language Processing | Jasamrit Rahala

---

Natural language processing or NLP is a wide branched area of AI that surrounds us everyday, from the autocomplete on your phones, to your searches on youtube. NLP is a blanket term that can relate to several AI techniques. We will focus on three of these techniques in this notebook:

- Markov Chains [Text generation]
- Naive Bayes [Text classification]
- Text preprocessing [Stop words, lemmatization, N-Grams, TF-IDF]

We will be using a minimum number of libraries to reinforce the maths and processes going behind the scenes, hopefully demistifying NLP. __Please work through this notebook at your own pace and ask any questions you may have on the discord.__ 

Github (useful links + resources) : https://github.com/JRahala

Thanks Jam

## Markov Chains | Text generation
---
Markov chains are based on the idea of dependent states. That is to say, __my current state is entirely dependent on the previous state I was in__. This idea is known as the __markov property__. Any system which can be modelled using this restraint can be modelled as a __markov chain__.

A __markov chain__ is a collection of states and probabilities of moving from one state to another. A very basic example of a markov chain could be the weather. The weather can either be raining or not. In this case there are two states. Now that we have established the states, we can assign transfer probabilities. 

- Assuming that the weather is raining we have a 10% chance of moving to a non-raining state and a 90% chance of continuing to rain. 
- Assuming that the weather is not raining we have a 40% chance of moving to a raining state and a 60% chance of continuing to not rain

Below is the following markov chain visualised

![Image](https://raw.githubusercontent.com/JRahala/tempStorage/master/Raining%20demo.png)


Ok so 'How does this relate to text __generation__?' you may ask. Since we have probabilities and states, we can start in any given state and walk along a path for n words or however long we want to until we reach a stop condition. Let's build a basic model of a markov chain using some probabilities for weather and build a simple weather forecaster.


### Weather forecasting | Basic example
---

In the example below I define weather as being in one of three states: [sunny, raining, cloudy]. Each state has a likelihood of trainsferring from one state to the next. This information is represented as 2D array, known as a transfer matrix. The first line represents __transfers from__ a sunny state, the second represents a raining state, the last represents a cloudy state. Going from left to right we are representing a __transfer to__ the sunny, raining and cloudy states accordingly. For example there is a 20% chance of going from a sunny to a cloudy state. i.e. row 0, col 2 or data[0][2]


The walking algorithm is reasonably simple. We use our current state (by default the sunny state) and generate a random number which is used to pick the next state. The __random.choices() function allows us to do choose random items in a array given weights__. Syntax is random.choices(array_to_choose_from, weights = array_of_weights, k = number_of_items_picked)[0]. Note that this returns a tuple since k will repeat this process (without affecting probabilities), therefore the subscript[0] is added to unpack the tuple to an integer.


In [1]:
import random

data = [    
    [50, 30, 20], # weather is sunny
    [20, 10, 70], # weather is raining
    [45, 5, 40]] # weather is cloudy

states = ['sunny', 'raining', 'cloudy']

current_state = 0
for day in range(10):
    current_state = random.choices([0,1,2], weights = data[current_state], k = 1)[0]
    
    print(f'Day {day} | weather - {states[current_state]}...')

Day 0 | weather - sunny...
Day 1 | weather - sunny...
Day 2 | weather - raining...
Day 3 | weather - cloudy...
Day 4 | weather - sunny...
Day 5 | weather - sunny...
Day 6 | weather - sunny...
Day 7 | weather - sunny...
Day 8 | weather - raining...
Day 9 | weather - sunny...


Now that we understand how a markov chain can be used, how can we generate one? (nevermind using text!)

### Markov Chains | Importing data
---

Inorder to train a Markov Chain we will need some data. For the purposes of demonstration I am using the book 'Jane Eyre'. The data was taken from the Project Gutenberg website https://www.gutenberg.org/files/1260/1260-h/1260-h.htm. To use this data I am going to use the requests library. If you don't understand how requests work, do not worry (in short it is simply requesting data from a website, I have hosted parts of the book on github so that I can easily request the data using python).

In [2]:
# import the requests module and set the url we want to retrieve data from

import requests
import random

url = 'https://raw.githubusercontent.com/JRahala/tempStorage/master/JaneEyre.txt'


# request the data and print of the first 100 characters of the data

data = requests.get(url).text
data[:100]

'CHAPTER I\r\nThere was no possibility of taking a walk that day.  We had been wandering, indeed, in th'

Above you can see the first 100 characters of Jane Eyre, you might notice the strange \n and \r marks. These are for formatting text so we can get rid of these by using the replace function.

In [3]:
data = data.replace('\n', ' ')
data = data.replace('\r', ' ')

data[:100]

'CHAPTER I  There was no possibility of taking a walk that day.  We had been wandering, indeed, in th'

Now that the strange marks have disappeared we can start with creating our Markov Chain. In this chain we will be using letters as a state and every succeeding letter will act as a possible nextState for the current letter. In this example I will use an object oriented approach (rather than a matrix like before) so let's set up a state class.

### Makov Chains | State Class
---

Our state class will need the following functionality:

- __init__()
- __update__(nextState) 
- __choose__() 

States require a value to store the letter and a dictionary containing the [new state] and probability of moving to that state

In [4]:
class State:
    
    Instances = {}
    
    def __init__(self, letter):
        
        State.Instances[letter] = self
        
        self.letter = letter
        self.links = {}
        
    def update(self, newState):
        
        if not newState in self.links:
            self.links[newState] = 0
            
        self.links[newState] += 1
    
    def choose(self):
        
        choice = random.random()
        
        probabilities = list(self.links.values())
        words = list(self.links.keys())

        return random.choices(words, probabilities, k = 1)[0]
        

A = State('a')
B = State('b')
C = State('c')

B.update('c')
B.update('c')
B.update('a')

print(B.links)


{'c': 2, 'a': 1}


Above I have created 3 states relating to the letters 'abc'. 

#### Updating states

I updated the B state with the A and the C states. When I update states I update the dictionary stored within each state object. When B is updated with state A, the key relating to A, is increased by 1. If no key exists it is set to 0 and then increased by 1. 

#### Choosing the next state

When we are choosing are next state from a current state, we use the random.choices function to make a random weighted selection. For example the chances of choosing C after being in state B are twice as large as moving to state A. This is because we updated B with state C twice whereas we only update B with state A once. So B.links looks like {'c': 2, 'a': 1}. By using the keys and values as arguments for the random.choices function we can choose C as the next state 2/3 times and choose A as the next state 1/3 times. Below I test out the the ratios returned by the choose function

In [5]:
Cs = 0
As = 0

for i in range(1000):
    if 'c' == B.choose(): Cs += 1
    else: As += 1

print(Cs, As)

661 339


As expected, C was chosen twice as much as A as the next state, this verifys our choose function.

### Markov Chains | Linking this together
---

Now that we have established a simple state class we can begin to use the data we have stored. First we split the data up into individual letters. Create states using those letters and update them with the relevant succeeding letters. For example in the sentence, 'A boy wore a hat'. 

- We create the 'A' state and update it with a ' '
- create the ' ' state and update it with a 'b'
- then create the 'b' class and update it with a 'o' 

and so on.

In [6]:
# split up the data letter by letter, for the sake of complexity I will also transform all letters into lowercase
data = list([x.lower() for x in data])
data.append('.')

# iteratively go through the data
for i in range(len(data) - 1):

    if not data[i] in State.Instances:
        State(data[i])
        
    currentState = State.Instances[data[i]]
    currentState.update(data[i+1])

Above we have created the Markov Chain objects and updated each object with the next letter. We can now do some iteration to produce some output based on a starting node. I arbitrarily chose 'a'.

In [7]:
currentNode = 'a'
depth = 100

while depth>0 and currentNode in State.Instances:
    currentNode = State.Instances[currentNode]
    currentNode = currentState.choose()
    print(currentNode, end = '')
    depth -= 1

 ay ab t  cc   ssa  tw  tlo  l   g iytgit siw  slsstsdal i fsnanw myp  rl  fi mtsiiwiwwl  h(iow ittw

The liklihood is that you will get some garbage produced above. This is because our model doesnt have enough context when choosing the next letter to go to. To counter this we will use something known as n-grams. Note: __It might be worth experimenting with words as states rather than letters__

### Markov Chains | N-Grams
---

N-Gram allow our model to have more context by using n-letters as a state, opposed to one letter. We were simply using unigrams (n = 1). This doesn't provide our model with much context. We can experiment with different values for n. Below I have reformated the code to accept any n.

In [8]:
State.Instances = {}

def train(data, n = 3):
    
    data.append('.')
    for i in range(len(data) - 1 - n):
        
        n_gram = ''.join(data[i:i+n])
        
        if not n_gram in State.Instances:
            State(n_gram)

        currentState = State.Instances[n_gram]
        currentState.update(data[i+n])
        
        
train(data, 10)
current_key = random.choice(list(State.Instances.keys()))


print('START:', current_key, end = '')
depth = 100

while current_key in State.Instances and depth > 0:
    
    currentState = State.Instances[current_key]
    new_letter = currentState.choose()
    
    print(new_letter, end = '')
    
    current_key = current_key[1:] + new_letter
    depth -= 1

START: -an-hour we never spoke; neither of you know where i was; gateshead and its daily luxuries.    chapter xxxvii 

Hopefully when you run the above code, you should see a comprehensible sentence. By changing the depth and n, you will get varying degrees of success. You may wonder __why cant I set really large if it gives my model more context?__. This is a fair point, however too much context just becomes too specific. If I am scanning the book for a 50 character long string, then there will only be so many exact 50 character strings until you just start to repeat the novel, word for word. 

By varying training data you can get very different and rather funny sentences produced in the style of a certain author. Text generation is an amusing use case however there are many other uses for markov chains including:

- analysing stocks
- generating music 
- genetic networks
- DNA sequences

Markov chains in the academic fields are used to simulate or study 'random' or stochastic processes and can model practically anything that fits the Markov property. __The probability of event (x) happening is instantaneous (doesnt depend on previous states)__. Of course this isn't entirely true however we can model such systems fairly accurately with a quick processing time. __Perhaps, as an extension, you could try and replace the text with stock prices for companies or incorporate more advanced text processing techniques into the chain?__

## Naive Bayes
---

Naive bayes classification is based on the bayes theorem, defined below. This is the formula for conditional probability. i.e. the probability of an event based on the prior knowledge of the conditions that might be related to the event.

<br>

$$ P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)} $$

<br>

$$ \frac{P(B|A) \cdot P(A)}{P(A) \cdot P(B|A) + P( \neg A) \cdot P(B| \neg A)} $$

<br>

By using this theorem we can calculate the probabilities of text data being a member of a certain class, in our case this means whether a news article is under: tech, business, sport, entertainment or politics. For this model I will be using a dataset found under here https://raw.githubusercontent.com/JRahala/tempStorage/master/bbc-text-short.csv

### Naive Bayes | Importing data
---

Once again I am using the requests module to retrive my data, since my data is a csv I will have to parse the data retrieved a little. This step is not crucial to understand.

In [9]:
import requests
url = 'https://raw.githubusercontent.com/JRahala/tempStorage/master/bbc-text-short.csv'

data = requests.get(url).text
result = []

for line in data.splitlines():
    result.append(tuple(line.split(",")))
    
headings = result[0]
result = result[1:]

print(headings)
print((result[0][0], result[0][1][:50]))

('category', 'text')
('tech', 'tv future in the hands of viewers with home theatr')


Now I have my data stored in the results array. We can start to create the NaiveBayes classifier class.

In [10]:
class NaiveBayes:
    
    def __init__(self):
        
        self.categories = {}
    
    def add_category(self, categoryName):
        
        self.categories[categoryName] = []
        
    def add_data(self, data):
        
        for example in data:
            
            category = example[0]
            article = example[1]
            
            if not category in self.categories:
                self.add_category(category)
                
            self.categories[category].append(article)
            

model = NaiveBayes()
model.add_data(result)
model.categories.keys()

dict_keys(['tech', 'business', 'sport', 'entertainment', 'politics'])

Above I am adding some training data to the classifier. You can go ahead and split the data into training and testing data if you want, it shouldn't make too much of a difference, given the nature of naive bayes classifiers. In the add_category function, I am simply creating an array for each category that is new. This array will hold all the articles for that specific category. I check the categories that my Naive Bayes has at the bottom. 

### Probabilities
---

Now that the basic skeleton of the model is set up we can start adding in the probability functions. In case you were confused with the whole Naive Bayes mathematics, here is a very useful video (that I based my explanation of) that can hopefully explain the bayes theorem in a simple way.

https://www.youtube.com/embed/HZGCoVF3YvM

Below I just calculate the probablity of an article being of a specific category. We just count how many articles exists and how many articles are of each category. We turn these numbers into proportions and we are done.

In [11]:
def p_category(self, category):
    
    return len(self.categories[category]) / sum(len(self.categories[category]) for category in self.categories)

NaiveBayes.p_category = p_category

probability_given_category = [(category, model.p_category(category)) for category in list(model.categories.keys())]
print(probability_given_category)

sum(x[1] for x in probability_given_category)

[('tech', 0.17617617617617617), ('business', 0.23523523523523523), ('sport', 0.23623623623623624), ('entertainment', 0.17217217217217218), ('politics', 0.18018018018018017)]


1.0

Above you can see that the probabilities sum up to 1 therefore the algorithm works. Now we can work out the probability of a word being in a certain category. We do this by counting up the number of unique words in that category and the number of times our word appears in this category's dataset.

In [12]:
def word_given_category(self, word, category):
    
    all_words = sum(len(x.split()) for x in self.categories[category])
    matches = sum(x.count(word) for x in self.categories[category])
    
    return matches / all_words

NaiveBayes.word_given_category = word_given_category # add this method to our NaiveBayes class

Now we can get the probability of a specific word appearing in a given category. I should note something about data processing. An example is shown below.

In [13]:
probability_given_category = [(category, model.word_given_category('tv', category)) for category in list(model.categories.keys())]
print(probability_given_category)

probability_given_category = [(category, model.word_given_category('TV', category)) for category in list(model.categories.keys())]
print(probability_given_category)

[('tech', 0.0021596580354944833), ('business', 0.00012672504467057825), ('sport', 6.288833547153674e-05), ('entertainment', 0.0018524699599465956), ('politics', 0.00011532296195495485)]
[('tech', 0.0), ('business', 0.0), ('sport', 0.0), ('entertainment', 0.0), ('politics', 0.0)]


When I run the cell above, I am requesting the probability of the phrase 'TV' given the categories and the probability of the phrase 'tv' given the categories. The results are printed above. When I used the lowercase 'tv' I recieved reasonable probabilities for each category however when I use the uppercase 'TV' all probabilities are set to 0. This means that 'TV' never appears in our data. This problem can spill over into other words, therefore it is best for us to transform all our data into lowercase, to make it easier to work with.

In [14]:
data = [(x[0], x[1].lower()) for x in result] # lower case data

We can now construct the whole categorise method. By looping over all the possible categories we can use the bayes theroem and calculate the product of the word_given_category() * p_category() for each word. This will produce numbers for each categories, we can turn these numbers into probabilities by dividing by the sum of all probabilities for each category. Below is the function.

In [15]:

def categorise(self, phrase):
    
    probabilities = {}
    
    for category in list(self.categories.keys()):
        probabilities[category] = 1
        
        for word in phrase.split():
            probabilities[category] *= self.word_given_category(word, category) * self.p_category(category)
        
        print(f'FINISHED : {category}')
        
    total_probability = sum(probabilities[category] for category in list(self.categories.keys()))
    
    for category in probabilities:
        probabilities[category] /= total_probability
        
    return probabilities


NaiveBayes.categorise = categorise

print('TEST DATA |', result[0][1][:200], '\n')
model.categorise(result[0][1][:200])

TEST DATA | tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in 

FINISHED : tech
FINISHED : business
FINISHED : sport
FINISHED : entertainment
FINISHED : politics


{'tech': 1.0,
 'business': 0.0,
 'sport': 0.0,
 'entertainment': 0.0,
 'politics': 0.0}

Above we test the algorithm by passing in the first sentence of our dataset. I used the first 200 characters, hence the [:200] because our algorithm is pretty inefficient (we dont store values, we calculate them on the fly). We can test the classifier using custom data as well, shown below.

In [16]:
model.categorise('technology is always changing and due to rapid development of the way we think about') # example data

FINISHED : tech
FINISHED : business
FINISHED : sport
FINISHED : entertainment
FINISHED : politics


{'tech': 0.8756354678743777,
 'business': 0.1232755550288938,
 'sport': 0.0,
 'entertainment': 0.0,
 'politics': 0.0010889770967286562}

In [17]:
model.categorise('m & s') # example data

FINISHED : tech
FINISHED : business
FINISHED : sport
FINISHED : entertainment
FINISHED : politics


{'tech': 0.048962803669511765,
 'business': 0.7013455722487103,
 'sport': 0.0348169203765133,
 'entertainment': 0.1667113638302523,
 'politics': 0.04816333987501238}

You might notice that if you test the classifier with characters that aren't included in the training dataset, then the algorithm crashes because of a division by zero error. To get past this error we can use laplace smoothing.

Laplace smoothing is described in this link. In terms of coding this means, when we calculate the word_given_category(), we add 1 to the numerator and add x to the denominator, where x = the number of unique words in the training data. You can try and implement a laplace smoothed version of word_given_category below.

In [18]:
def word_given_category(self, word, category):
    
    all_words = sum(len(x.split()) for x in self.categories[category])
    matches = sum(x.count(word) for x in self.categories[category])
    
    return matches / all_words # what can you add here !!!

NaiveBayes.word_given_category = word_given_category # add this method to our NaiveBayes class

# feel free to try out the new method with some test text

This sums up the naive bayes classification technique. Now that you understand how to create the python class, you have hopefully gained an understanding of how naive bayes is used and in what situations you could use this technique. As well as this you now know one of the most important theorems in probability, the bayes theorem. If you read on to the next chapter I should be going over an implementation of naive bayes using the sklearn library, this is something you would want to use in development or deployment.

## Processing and using Sklearn

Please have a look at this tutorial online to learn how to use sklearn to build upon your existing nlp knowledge.

https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

Thanks,
Jam